In [1]:
# import library
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random

2023-06-07 10:24:05.998056: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 10:24:06.820922: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
tf.keras.backend.set_floatx("float64")  # set all numbers to float_32bit

In [3]:
# known parameters declaration
m = 0.8
g = 9.81
l = 1.5
tau = 0.2

In [4]:
# data for training model: data for time t
num = 1600  # generate number of training points
time_train = 10
t_data = np.zeros((num, 1))  # array to store varying time t
for i in range (0, np.shape(t_data)[0]):
    t_data[i,0] = time_train*random.uniform(0, 1)

print(t_data) # print(t)  # print to check

#data for the 2 initial conditions:
tc = np.zeros((num, 1))  # array to store varying time t

[[8.32683002]
 [5.00374887]
 [1.96601303]
 ...
 [6.59468884]
 [0.85865555]
 [9.78510236]]


In [5]:
t_train = tf.concat([tc, t_data], 0)
t_train = tf.convert_to_tensor(t_train)

2023-06-07 10:24:10.434667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 10:24:10.632705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-07 10:24:10.632889: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
# customize Activation function
from keras import backend as K
def sine_actfn(x, beta=1):
    # return x * K.sin(beta*x)
    # return x + K.sin(beta*x)
    # return K.sin(beta*x)
    return (x - 1/(2*beta) * K.cos(2*beta*x) + 1/(2*beta)) 

In [7]:
from tensorflow import keras
# define the structure of the Model
def DNN_builder(in_shape, out_shape, n_hidden_layers, neuron_per_layer, actfn):
    # input layer
    input_layer = tf.keras.layers.Input(shape=(in_shape,))
    # max_pool_1d = tf.keras.layers.MaxPooling1D(pool_size=2)

    # hidden layers
    hidden = [tf.keras.layers.FNN(neuron_per_layer, activation=actfn, recurrent_activation=actfn, return_sequences=False,)(input_layer)]
    for i in range(n_hidden_layers-1):
        new_layer = tf.keras.layers.LSTM(neuron_per_layer, activation=actfn, recurrent_activation=actfn, return_sequences=False,)(hidden[-1])
        hidden.append(new_layer)

    # output layer
    output_layer = tf.keras.layers.LSTM(out_shape, activation=None)(hidden[-1])

    # building the model
    model = tf.keras.Model(inputs = input_layer, outputs = output_layer,)  # assign input, output layer and name of the network
    return model

model = DNN_builder(1, 1, 5, 32, sine_actfn)
model.summary()

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 1)

In [ ]:
@tf.function
def theta(t):  # construct the output, input 
    theta = model(t)
    return theta

In [ ]:
@tf.function
def f(t): # loss function in terms of the Equation itself
    u0 = theta(t)
    u_t = tf.gradients(u0, t)[0]  # du/dt
    u_tt = tf.gradients(u_t, t)[0]  # d2u/dt2

    F = m*(l**2)*u_tt - tau + m*g*l*tf.cos(u0)  # the function
    return tf.reduce_mean(tf.square(F))

In [ ]:
@tf.function
def u_dot(t): # compute u_dot
    u0 = theta(t)
    u_t = (1 - tf.sign(t)) * tf.gradients(u0, t)[0]  
    return u_t

In [ ]:
@tf.function
def u0(t): # compute u_dot
    u0 = (1 - tf.sign(t))*theta(t)
    return u0

In [ ]:
# MSE loss function of (estimated vs actual) value
@tf.function
def mse(y, y_):
    error = tf.abs(y-y_)
    # error = tf.square(y-y_)
    return tf.reduce_mean(error)

In [8]:
epochs = 4000
loss_list = np.array([])
loss = 0

# select Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=7e-4)

# training loop
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        y0 = u0(t_train)  # get ESTIMATED Output for y(0)
        y_dot0 = u_dot(t_train)   # get ESTIMATED Output for dy/dt(0)
        L1 = f(t_train)  # loss in terms of FUnctions itself
        L2 = mse(y0, 0)  # loss function in terms of Output Condition 1st: y(0)
        L3 = mse(y_dot0, 0) # loss function in terms of Output Condition 2nd: dy(0)/dt = 0
        loss = 2*L1 + L2 + L3

    # compute gradients
    g = tape.gradient(loss, model.trainable_variables)
    loss_list = np.append(loss_list, loss)
    # log every 200 epochs
    if (epoch % 200 == 0) or (epoch == epochs-1):
        print(f"epoch: {epoch:4}, loss = {loss.numpy():.4f}")
    # apply gradients
    opt.apply_gradients(zip(g, model.trainable_variables))


# plot the loss value in terms of log scale
plt.semilogy(range(epochs), loss_list)
plt.ylabel("loss")
plt.xlabel("epoch")

NameError: name 'u0' is not defined

In [9]:
# for i in range (1, 5):
#     w = model.layers[i].get_weights()

# print(w)

In [10]:
#Extract data from MATLAB file to plot graph
from scipy.io import loadmat
t_data = loadmat("t_1_dof.mat")
t_data = t_data.get('t')   # load data of t from .mat file---------------------------------

exact_sol = loadmat("out_1_dof.mat")
exact_sol = exact_sol.get('out_1_dof')   # load data of angle theta1 from .mat file------------

In [ ]:
# print(t_data.keys())
# print(t_data)
# print(exact_sol.keys())

In [ ]:
# plot exact result
plt.plot(t_data, exact_sol, '--k', label = "Actual", lw=2)
#plot predicted result
t_test = np.linspace(0, 22, 2000)
plt.plot(t_data, theta(t_data), label = "Predicted")

plt.title('1 DOF')
plt.xlabel('Time (s)')
plt.ylabel('Theta (rad)')
plt.legend()
plt.show()